In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
words = open('names.txt','r').read().splitlines()
words[:10]

In [ ]:
len(words)

In [ ]:
# build the vocab of chars and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s: i+1 for i,s in enumerate(chars)}
stoi['.'] = 0

itos = {i:s for s,i in stoi.items()}
itos

In [ ]:
# build the dataset
block_size = 3
X, Y = [], []

for w in words:
    # print(w)
    context = [0] * block_size
    
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(itos[i] for i in context), '---->', itos[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [ ]:
X.shape, X.dtype, Y.shape, Y.dtype

In [ ]:
# Lookup table
C = torch.randn((27,2))

In [ ]:
C[X][1,1]

In [ ]:
emb = C[X]
emb.shape

In [ ]:
# Middle layer
W1 = torch.randn((6,100))
b1 = torch.randn(100)

In [ ]:
emb[:, 0, :].shape

In [ ]:
torch.cat([emb[:,0,:], emb[:,1,:], emb[:,2,:]], 1).shape

In [ ]:
torch.cat(torch.unbind(emb, 1), 1).shape

In [ ]:
# more efficient
# -1 torch infers the correct dimension
emb.view(-1, 6).shape

In [ ]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [ ]:
# Final layer
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

logits = h @ W2 + b2

In [ ]:
logits.shape
counts = logits.exp()
prob = counts / counts.sum(1, keepdims=True)

In [ ]:
prob.shape

In [ ]:
prob[0].sum()

In [ ]:
loss = -prob[torch.arange(228146), Y].log().mean()
loss

In [ ]:
# --- Summary ----

In [ ]:
X.shape, Y.shape

In [ ]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)

parameters = [C, W1, b1, W2, b2]

In [ ]:
sum(p.nelement() for p in parameters)

In [ ]:
for p in parameters:
    p.requires_grad = True

In [ ]:
for i in range(1000):
    
    # forward pass
    emb = C[X] # (32, 3, 2)
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y)
    print(loss.item())
    
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    # update
    for p in parameters:
        p.data += -0.1 * p.grad


In [ ]:
# This takes too long though
# We can do the forward&backward pass on random batches of the training set